In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Homework 9
Let's predict the future.

## Part 0: Setup

In [2]:
import tensorflow as tf
import numpy as np
import util

# Colors to visualize the labeling
COLORS = np.array([(0,0,0), (255,0,0), (0,255,0), (255,255,0), (0,0,255), (255,255,255)], dtype=np.uint8)
CROP_SIZE = 64
N_ACTION = 5

offsets = [0, 6, 15, 30, 60, 120]

def parser(record):
    # Parse the TF record
    
    feature ={
        'height': tf.FixedLenFeature([], tf.int64),
        'width': tf.FixedLenFeature([], tf.int64),
        'channels': tf.FixedLenFeature([], tf.int64),
        'n_future': tf.FixedLenFeature([], tf.int64),
        'action': tf.FixedLenFeature([], tf.int64),
        'image_raw': tf.FixedLenFeature([], tf.string)
    }
    values = {'position' : np.float32, 'is_dying': np.int64, 'on_ground': np.int64, 'coins': np.int64}
    for k,v in enumerate(values):
        t = values[v]
        for j, o in enumerate(offsets):
            feature[v+'_%d'%j] = tf.FixedLenFeature([], t)

    parsed = tf.parse_single_example(record, features=feature)
     
    # Load the data and format it
    W = tf.cast(parsed['width'], tf.int32)
    H = tf.cast(parsed['height'], tf.int32)
    C = tf.cast(parsed['channels'], tf.int32)
    A = tf.cast(parsed['action'], tf.int32)
    actions = tf.stack([tf.bitwise.bitwise_and(A, (1<<i)) > 0 for i in range(N_ACTION)])
    image = tf.reshape(tf.decode_raw(parsed["image_raw"], tf.uint8), [H,W,C])
    
    current_position = tf.cast(parsed['position_0'], tf.float32)
    current_is_dying = tf.cast(parsed['is_dying_0'], tf.bool)
    current_coins = tf.cast(parsed['coins_0'], tf.float32)
    
    future_position = tf.stack([tf.cast(parsed['position_%d'%o], tf.float32) for o in range(1,len(offsets))])
    future_is_dying = tf.stack([tf.cast(parsed['is_dying_%d'%o], tf.bool) for o in range(1,len(offsets))])
    future_coins = tf.stack([tf.cast(parsed['coins_%d'%o], tf.float32) for o in range(1,len(offsets))])
    
    ## No data augmentation this time, as it might affect the future
    return A, image, actions, current_position, current_is_dying, current_coins, future_position, future_is_dying, future_coins

def load_dataset(tfrecord):
    # Load the dataset
    dataset = tf.contrib.data.TFRecordDataset(tfrecord)

    # Parse the tf record entries
    dataset = dataset.map(parser, num_threads=8, output_buffer_size=1024)

    # Shuffle the data, batch it and run this for multiple epochs
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(32)
    dataset = dataset.repeat()
    return dataset

## Part 1: Define your network

In [3]:
# Create a new log directory (if you run low on disk space you can either disable this or delete old logs)
# run: `tensorboard --logdir log` to see all the nice summaries
for n_model in range(1000):
    LOG_DIR = 'log/model_%d'%n_model
    from os import path
    if not path.exists(LOG_DIR):
        break

# Lets clear the tensorflow graph, so that you don't have to restart the notebook every time you change the network
tf.reset_default_graph()

TF_COLORS = tf.constant(COLORS)

train_data = load_dataset('future_train.tfrecord')
valid_data = load_dataset('future_val.tfrecord')

# Create an iterator for the datasets
# The iterator allows us to quickly switch between training and validataion
iterator = tf.contrib.data.Iterator.from_structure(train_data.output_types, ((None,), (None,64,64,9), (None,N_ACTION), (None,), (None,), (None,), (None,len(offsets)-1), (None,len(offsets)-1), (None,len(offsets)-1)))

# and fetch the next images from the dataset (every time next_image is evaluated a new image set of 32 images is returned)
A, image, action, current_position, current_is_dying, current_coins, future_position, future_is_dying, future_coins = iterator.get_next()

# Define operations that switch between train and valid
switch_train_op = iterator.make_initializer(train_data)
switch_valid_op = iterator.make_initializer(valid_data)

# Convert the input and label
image = tf.identity(image, name='images')
print(image)
image = tf.cast(image, tf.float32)
action = tf.identity(tf.cast(action, tf.int32), name='action')

current_position = tf.identity(current_position, name='current_position')
current_is_dying = tf.identity(current_is_dying, name='current_is_dying')
current_coins = tf.identity(current_coins, name='current_coins')

future_position = tf.identity(future_position, name='future_position')
future_is_dying = tf.identity(future_is_dying, name='future_is_dying')
future_coins = tf.identity(future_coins, name='future_coins')

print(action)
print(image)
print(action)
print(current_position)
print(current_is_dying)
print(current_coins)
print(future_position)
print(future_is_dying)
print(future_coins)

# Whiten the image
white_image = (image - 100.) / 72.

# TODO: Define your convnet and loss here
# In preparation for the next homework you might want to make this network small and efficient.

# Build the network out of a few convolutional layers
h = white_image
for i, n in enumerate([10,20,30,40,50,60]):
    h = tf.contrib.layers.conv2d(h, 20, (3,3), stride=2, scope="conv%d"%i)
h = tf.contrib.layers.flatten(h)

# Hook up a fully connected layer to predict the action
action_logit = tf.contrib.layers.fully_connected(h, N_ACTION, activation_fn=None)

# Combine the action and the output of the conv layer
outer_product = tf.contrib.layers.flatten( tf.cast(action[:,None,:], tf.float32) * h[:,:,None] )
h = tf.contrib.layers.fully_connected(outer_product, 200)
h = tf.contrib.layers.fully_connected(h, 100)
future_position_logit = tf.contrib.layers.fully_connected(h, len(offsets)-1, activation_fn=None) + current_position[:,None]
future_is_dying_logit = tf.contrib.layers.fully_connected(h, len(offsets)-1, activation_fn=None) + tf.cast(current_is_dying[:,None], tf.float32)-0.5
future_coins_logit = tf.contrib.layers.fully_connected(h, len(offsets)-1, activation_fn=None) + current_coins[:,None]

# Use a cross entropy for the action and is_dying, L2 for position and coin.
action_loss = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.cast(action, tf.float32), logits=action_logit) )
is_dying_loss = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.cast(future_is_dying, tf.float32), logits=future_is_dying_logit) )
position_loss = tf.reduce_mean( tf.nn.l2_loss(future_position_logit-future_position) )
coins_loss = tf.reduce_mean( tf.nn.l2_loss(future_coins_logit-future_coins) )
loss = 0.05*coins_loss + position_loss + is_dying_loss + action_loss

# a binary vector of size 5 for each image in the batch (DO NOT USE action TO PREDICT THIS!)
pred_action = tf.identity(action_logit > 0.5, name='predicted_action')

# vectors of size (5) for each image in the batch (USE action AND current_position, current_is_dying, current_coins TO PREDICT THIS)
# Hint for some variables you might want to make them relative to current_..., for others not
pred_is_dying = tf.identity(future_is_dying_logit > 0.5, name='predicted_is_dying')
pred_position = tf.identity(future_position_logit, name='predicted_position')
pred_coins = tf.identity(future_coins_logit, name='predicted_coins')

# Accuracies
action_acc = tf.reduce_mean(tf.cast(tf.equal(tf.cast(pred_action, tf.float32), tf.cast(action, tf.float32)), tf.float32))
dying_acc = tf.reduce_mean(tf.cast(tf.equal(tf.cast(pred_is_dying, tf.float32), tf.cast(future_is_dying, tf.float32)), tf.float32))

# Let's weight the regularization loss down, otherwise it will hurt the model performance
# You can tune this weight if you wish
regularization_loss = tf.losses.get_regularization_loss()
total_loss = loss + 1e-6 * regularization_loss

# Adam will likely converge much faster than SGD for this assignment.
optimizer = tf.train.AdamOptimizer(0.001, 0.9, 0.999)

# use that optimizer on your loss function (control_dependencies makes sure any 
# batch_norm parameters are properly updated)
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    opt = optimizer.minimize(total_loss)

# Let's define some summaries for tensorboard
tf.summary.image('image1', image[:,:,:,:3], max_outputs=3)
tf.summary.image('image2', image[:,:,:,3:6], max_outputs=3)
tf.summary.image('image3', image[:,:,:,6:9], max_outputs=3)
tf.summary.scalar('action_loss', tf.placeholder(tf.float32, name='action_loss'))
tf.summary.scalar('is_dying_loss', tf.placeholder(tf.float32, name='is_dying_loss'))
tf.summary.scalar('position_loss', tf.placeholder(tf.float32, name='position_loss'))
tf.summary.scalar('coins_loss', tf.placeholder(tf.float32, name='coins_loss'))
tf.summary.scalar('loss', tf.placeholder(tf.float32, name='loss'))
tf.summary.scalar('val_loss', tf.placeholder(tf.float32, name='val_loss'))

merged_summary = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter(LOG_DIR, tf.get_default_graph())

# Let's compute the model size
print( "Total number of variables used ", np.sum([v.get_shape().num_elements() for v in tf.trainable_variables()]) )

Tensor("images:0", shape=(?, 64, 64, 9), dtype=uint8)
Tensor("action:0", shape=(?, 5), dtype=int32)
Tensor("Cast:0", shape=(?, 64, 64, 9), dtype=float32)
Tensor("action:0", shape=(?, 5), dtype=int32)
Tensor("current_position:0", shape=(?,), dtype=float32)
Tensor("current_is_dying:0", shape=(?,), dtype=bool)
Tensor("current_coins:0", shape=(?,), dtype=float32)
Tensor("future_position:0", shape=(?, 5), dtype=float32)
Tensor("future_is_dying:0", shape=(?, 5), dtype=bool)
Tensor("future_coins:0", shape=(?, 5), dtype=float32)
Total number of variables used  61660


## Part 2: Training

Training might take up to 20 min depending on your architecture (and if you have a GPU or not).

In [4]:
# Start a session
sess = tf.Session()

# Set up training
sess.run(tf.global_variables_initializer())

# Run the training for some iterations
for it in range(10):
    sess.run(switch_train_op)

    loss_vals = []
    # Run 10 training iterations and 1 validation iteration
    for i in range(100):
        loss_val = sess.run([loss,action_loss,is_dying_loss,position_loss,coins_loss,opt])[:-1]
        loss_vals.append(loss_val)
    # Compute the summary
    mean_loss = np.mean(np.array(loss_vals), axis=0)
    summary = {n+':0':mean_loss[i] for i, n in enumerate(['loss','action_loss','is_dying_loss','position_loss','coins_loss'])}

    # Compute the validation loss
    sess.run(switch_valid_op)
    loss_val = sess.run(loss)
    summary['val_loss:0'] = loss_val
    
    summary_writer.add_summary( sess.run(merged_summary, summary), it )

    # Let's update tensorboard
    print('[%3d] Loss: %0.3f  \t  val loss A.: %0.3f'%(it, mean_loss[0], loss_val))


[  0] Loss: 0.796  	  val loss A.: 0.895
[  1] Loss: 0.522  	  val loss A.: 0.422
[  2] Loss: 0.454  	  val loss A.: 0.339
[  3] Loss: 0.408  	  val loss A.: 0.355
[  4] Loss: 0.399  	  val loss A.: 0.457
[  5] Loss: 0.393  	  val loss A.: 1.119
[  6] Loss: 0.393  	  val loss A.: 0.323
[  7] Loss: 0.380  	  val loss A.: 0.350
[  8] Loss: 0.384  	  val loss A.: 0.367
[  9] Loss: 0.371  	  val loss A.: 0.417


## Part 3: Evaluation
### Compute the validation accuracy

In [5]:
action_p = tf.placeholder(tf.int32, shape=(None))
dying_p = tf.placeholder(tf.bool, shape=(None))
pos_p = tf.placeholder(tf.float32, shape=(None))
coins_p = tf.placeholder(tf.float32, shape=(None))

#####################
action_accs = []
dying_accs = []
pos_losses = []
coin_losses = []
index = 0
for it in tf.python_io.tf_record_iterator('future_val.tfrecord'):
    index += 1
    if index % 10 == 0:
        print(index)
    example = tf.train.Example()
    example.ParseFromString(it)
    
    # Get ground truth inputs
    I = np.expand_dims(np.frombuffer(example.features.feature['image_raw'].bytes_list.value[0], dtype=np.uint8).reshape(64, 64, 9), axis=0)
    A = example.features.feature['action'].int64_list.value[0]
    action = tf.stack([tf.bitwise.bitwise_and(action_p, (1<<i)) > 0 for i in range(N_ACTION)])
    current_position = [example.features.feature['position_0'].float_list.value[0]]
    current_is_dying = [example.features.feature['is_dying_0'].int64_list.value[0]]
    current_coins = [example.features.feature['coins_0'].int64_list.value[0]]
    
    # Get ground truth labels
    val_future_pos = np.expand_dims(np.array([example.features.feature['position_%d'%o].float_list.value[0] for o in range(1,len(offsets))]), axis=-1)
    val_future_dying = np.expand_dims(np.array([example.features.feature['is_dying_%d'%o].int64_list.value[0] for o in range(1,len(offsets))]), axis=-1)
    val_future_coins = np.expand_dims(np.array([example.features.feature['coins_%d'%o].int64_list.value[0] for o in range(1,len(offsets))]), axis=-1)
    
    future_position = tf.stack(tf.cast(pos_p, tf.float32))
    future_is_dying = tf.stack(tf.cast(dying_p, tf.float32))
    future_coins = tf.stack(tf.cast(coins_p, tf.float32))
    
    action, future_position, future_is_dying, future_coins = sess.run([action, future_position, future_is_dying, future_coins], {action_p: A, pos_p: val_future_pos, dying_p: val_future_dying, coins_p: val_future_coins})
    action = np.expand_dims(np.array(action), axis=0)
    
    # Make predictions
    action_pred, pos_pred, dying_pred, coins_pred = sess.run([tf.cast(pred_action, tf.float32), future_position_logit, tf.cast(pred_is_dying, tf.float32), future_coins_logit], {'images:0': I, 'action:0': action, 'current_position:0': current_position, 'current_is_dying:0': current_is_dying, 'current_coins:0': current_coins})
    action_pred = action_pred[0]
    pos_pred = pos_pred[0]
    dying_pred = dying_pred[0]
    coins_pred = coins_pred[0]
    
    # Score predictions
    val_action_acc = np.mean((action_pred == action).astype('float32'))
    val_dying_acc = np.mean((dying_pred == future_is_dying).astype('float32'))
    val_pos_acc = np.mean((pos_pred - future_position) ** 2)
    val_coin_acc = np.mean((coins_pred - future_coins) ** 2)
    
    action_accs.append(val_action_acc)
    dying_accs.append(val_dying_acc)
    pos_losses.append(val_pos_acc)
    coin_losses.append(val_coin_acc)
    
print('Action accuracy: ' + str(np.mean(action_accs)))
print('Is-Dying accuracy: ' + str(np.mean(dying_accs)))
print('Position L2: ' + str(np.mean(pos_losses)))
print('Coin L2: ' + str(np.mean(coin_losses)))


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
Action accura

## Part 4: Save Model
Please note that we also want you to turn in your ipynb for this assignment.  Zip up the ipynb along with the tfg for your submission.

In [8]:
util.save('assignment9.tfg', session=sess)

In [9]:
tf.__version__

'1.3.0'